In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install --upgrade-strategy eager "optimum[openvino,nncf]"

In [3]:
import torch
import pandas as pd

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from evaluate import evaluator
import evaluate

#**Сonvert to OpenVino**

In [4]:
from optimum.intel import OVModelForSequenceClassification

model_checkpoint = "lvwerra/distilbert-imdb"
save_directory = "ov/"

# Load a model from transformers and export it to OV
ov_model = OVModelForSequenceClassification.from_pretrained(model_checkpoint, export=True)
#tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ov_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


Framework not specified. Using pt to export to ONNX.


Using framework PyTorch: 2.1.0+cu118
/usr/local/lib/python3.10/dist-packages/nncf/torch/dynamic_graph/wrappers.py:74: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  op1 = operator(*args, **kwargs)
Compiling the model...
Set CACHE_DIR to /tmp/tmpumzltgik/model_cache


('ov/tokenizer_config.json',
 'ov/special_tokens_map.json',
 'ov/vocab.txt',
 'ov/added_tokens.json',
 'ov/tokenizer.json')

In [5]:
clf_metrics = evaluate.combine(["accuracy",
                                "f1",
                                "precision",
                                "recall"])
cpu_pipe = pipeline("text-classification",
                model=ov_model,
                tokenizer=tokenizer,
                #device='cuda' на cuda не завелось
                )

data = load_dataset("imdb",
                    split="test").shuffle().select(range(1000))
task_evaluator = evaluator("text-classification")

cpu_results = task_evaluator.compute(model_or_pipeline=cpu_pipe,
                                 data=data,
                                 metric=clf_metrics,
                                 label_mapping={"NEGATIVE": 0, "POSITIVE": 1},)
print('CPU RESULTS')
print(cpu_results)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

CPU RESULTS
{'accuracy': 0.947, 'f1': 0.9475766567754699, 'precision': 0.9618473895582329, 'recall': 0.9337231968810916, 'total_time_in_seconds': 454.9201137840001, 'samples_per_second': 2.1981881427094, 'latency_in_seconds': 0.4549201137840001}


результат в таблице https://docs.google.com/spreadsheets/d/1raJc55pfesADS5l5FFFuZklGGY3s23ml3yISpkk02g0/edit?usp=sharing